# Summary
Tests/shows usage of the joins module for calculating 
Statistics Canada access at 50/10 (or other levels) with custom geometry 
using the Pseudo Household (PHH) data. 

Produces no output files.

In [ ]:
import sys
sys.path.append("..")

import src.datasets.loading.statcan as statcan
import pandas as pd

In [ ]:
import src.datasets.joins as joins

In [ ]:
import src.config

In [ ]:
phh_speed_csv = src.config.DATA_DIRECTORY / 'PHH' / 'CRTC_Speed_Data' / 'PHH_Speeds_Current-PHH_Vitesses_Actuelles.csv'
phh_speeds = pd.read_csv(phh_speed_csv)
phh_speeds = phh_speeds.merge(statcan.phh_csv_data(), on='PHH_ID')
phh_speeds['Pop2016_at_50_10_combined'] = phh_speeds['Combined_50_10_Combine']*phh_speeds['Pop2016']
phh_speeds['TDwell_at_50_10_combined'] = phh_speeds['Combined_50_10_Combine']*phh_speeds['TDwell2016_TLog2016']

In [ ]:
phh_speeds

In [ ]:
phh_geo = statcan.phh_geometry()
phh_full = phh_geo.merge(phh_speeds, on='PHH_ID')

In [ ]:
phh_full = phh_full[phh_full.Type!=8]

In [ ]:
custom_geometry = joins.hexagons_popctrs_removed()

In [ ]:
custom_geometry

In [ ]:
phh_custom_join = phh_full.sjoin(custom_geometry[['geometry','HEXUID_PCPUID']])

In [ ]:
phh_custom_join.columns

In [ ]:
custom_aggs = phh_custom_join.groupby('HEXUID_PCPUID')[['Pop2016', 'TDwell2016_TLog2016', 'URDwell2016_RH2016','pop_at_50_10_combined', 'TDwell_at_50_10_combined',]].sum()

In [ ]:
custom_aggs.loc['480191']

In [ ]:
f'{1.237646e6:,.0f}'

In [ ]:
custom_aggs.loc['AB51361140-XXXXXX']

In [ ]:
grps = phh_speeds.groupby('HEXUID_IdUHEX')
manual_hexagons = pd.concat([
    grps['Pop2016'].sum(),
    grps['TDwell2016_TLog2016'].sum(),
    grps['URDwell2016_RH2016'].sum(),
    grps['pop_at_50_10_combined'].sum(),
    grps['TDwell_at_50_10_combined'].sum()
],axis=1)
manual_hexagons['percent_pop_50_10_avail'] = (manual_hexagons['pop_at_50_10_combined']/manual_hexagons['Pop2016']*100)
manual_hexagons['percent_dwell_50_10_avail'] = (manual_hexagons['TDwell_at_50_10_combined']/manual_hexagons['TDwell2016_TLog2016']*100)

In [ ]:
samples = manual_hexagons.sample(10).index

In [ ]:
manual_hexagons.loc[samples.values].sort_index()

In [ ]:
statcan.hexagon_data().loc[lambda s:s.HEXuid_HEXidu.isin(samples)].sort_values(by='HEXuid_HEXidu')

In [ ]:
phh_full